In [45]:
#Temizlik kodu
#Bu kod wpdeki excel alıp temizler, sütunları düzeltir küfürleri ve ingilizce ve saçma kelimeler içeren satırları siler.

import pandas as pd
import re
from collections import Counter
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter
#from langdetect import detect
import json


# TEST_PATH = "./constants/test.json"

# test = ""
# with open(TEST_PATH, "r") as f:
#     # read the json file
#     test = f.read()
#     # close the file
#     f.close()

# # print the test file
# print("Test: ", test)

# Langdetect sonuçlarının tutarlılığı için tohum ayarı
#DetectorFactory.seed = 0

# Dosya yolları
INPUT_PATH = "../dataset/reviews.csv"
OUTPUT_PATH = "../dataset/cleaned_reviews.xlsx"


# KEYWORDS
SWEAR_WORDS_PATH = "./constants/swear_words.json"
ASPECTS_KEYWORDS_PATH = "./constants/aspects_keywords.json"
SENTIMENT_KEYWORDS_PATH = "./constants/sentiment_keywords.json"
COLORS_MAP_PATH = "./constants/colors_map.py"

# import swear words from swear_words.py file
# read and add the swear words to a list
SWEAR_WORDS = []
ASPECTS_KEYWORDS = []
SENTIMENT_KEYWORDS = []

from openpyxl.styles import PatternFill
COLOR_MAP = {
    "Olumlu": PatternFill(start_color="90EE90", fill_type="solid"),
    "Olumsuz": PatternFill(start_color="FFA07A", fill_type="solid"),
    "Nötr": PatternFill(start_color="D3D3D3", fill_type="solid"),
}


In [ ]:
# read json file
def read_json(file_path):
    with open(file_path, "r", encoding='utf-8') as f:
        # read the json file
        data = json.load(f)
        # close the file

        f.close()
    return data


In [49]:
# GET THE SWEAR WORDS FROM THE SWEAR WORDS FILE
swear_data = read_json(SWEAR_WORDS_PATH)
SWEAR_WORDS = swear_data["SWEAR_WORDS"]

# GET THE ASPECTS KEYWORDS FROM THE ASPECTS KEYWORDS FILE
aspects_data = read_json(ASPECTS_KEYWORDS_PATH)
ASPECTS_KEYWORDS = aspects_data["ASPECTS_KEYWORDS"]

# GET THE SENTIMENT KEYWORDS FROM THE SENTIMENT KEYWORDS FILE
sentiments_data = read_json(SENTIMENT_KEYWORDS_PATH)
SENTIMENT_KEYWORDS = sentiments_data["SENTIMENT_KEYWORDS"]

# print the swear words
print("Swear words: ", SWEAR_WORDS)
# print the aspects keywords
print("Aspects keywords: ", ASPECTS_KEYWORDS)
# print the sentiment keywords
print("Sentiment keywords: ", SENTIMENT_KEYWORDS)

Swear words:  ['amk', 'a.m.k', 'mk', 'aq', 'mq', 'am n k', 'am q', 'amq', 'amına koyayım', 'amınakoyayım', 'amına koyim', 'amına koyyim', 'a m k', 'a.m.k.', 'amınoğlu', 'amınoglu', 'anasikim', 'oc', 'o.c', 'oç', 'o.ç', 'siktir', 's.iktir', 'sik tir', 's iktir', 'sikti', 's.ikti', 'siktir lan', 's.iktir lan', 'siktir git', 's.iktir git', 'sikeyim', 's.ikeyim', 'skrm', 's.krm', 'sk.rm', 'sik', 's.ik', 'sikim', 's.ikim', 'sikmiş', 's.ikmiş', 'sikim gibi', 's.ikim gibi', 'sikim gibi', 's.ikim gibi', 'bok', 'boktan', 'bok gibi', 'bokbok', 'bokçuluk', 'yavşak', 'yavsak', 'sürtük', 'sürtüğün', 'gerizekalı', 'aptal', 'salak', 'orospu', 'orospu', 'orospu evladı', 'orospu çocuğu', 'oç', 'o.ç', 'o rospu', 'orospunun', 'o rospu çocuğu', 'porno', 'p0rn0', 'porn', 'p0rn', 'sürtük', 'sürtükler', 'sürtük gibi', 'sürtükler gibi', 'sürtükçü', 'sürtükçüler', 'sürtükçü gibi', 'kahpe', 'k.a.h.p.e', 'kah.pe', 'kahpe evladı', 'kahpe çocuğu', 'kahpe gibi', 'kahpe gibi', 'sikiş', 's.ikiş', 'sikis', 'sikmiş', '

In [50]:
SWEAR_WORDS_PATTERNS = re.compile("|".join(map(re.escape, SWEAR_WORDS)), re.IGNORECASE)

In [51]:
# --- FONKSIYONLAR ---
def normalize_text(text):
    replacements = str.maketrans("çğıöşü", "cgiosu")
    return text.translate(replacements)



def has_keyword_with_suffix(text, keyword):
    suffixes = [
        "", "i", "ı", "u", "ü", "e", "a", "de", "da", "den", "dan", "ye", "ya", "nin", "nın", "nun", "nün",
        "ne", "na", "ler", "lar", "si", "sı", "su", "sü", "inden", "undan", "lerinin", "larının", "deki",
        "daki", "yle", "yla", "li", "lı", "lu", "lü", "ik", "ık", "ük", "ek", "ak"
    ]
    keyword = normalize_text(keyword.lower())
    text = normalize_text(text.lower())
    pattern = r'\b' + f"{keyword}({'|'.join(suffixes)})?" + r'\b'
    return re.search(pattern, text) is not None



def has_extended_match(text, keyword):
    text = normalize_text(text.lower())
    keyword = normalize_text(keyword.lower())
    pattern = r'\b\w*' + re.escape(keyword) + r'\w*\b'
    matches = re.findall(pattern, text)
    for match in matches:
        if len(match) >= len(keyword) + 2:
            return True
    return keyword in text




def keyword_match(text, keyword):
    return has_keyword_with_suffix(text, keyword) or has_extended_match(text, keyword)




def detect_sentiment(text, aspect_keyword):
    try:
        text = str(text).lower()
        aspect_keyword = aspect_keyword.lower()
        sentences = re.split(r'[.!?]', text)
        target_sentences = [sentence.strip() for sentence in sentences if keyword_match(sentence, aspect_keyword)]
        if not target_sentences:
            return "Nötr"

        context_text = " ".join(target_sentences)
        words = re.findall(r'\b\w+\b', normalize_text(context_text.lower()))
        pos = [normalize_text(w) for w in SENTIMENT_KEYWORDS["Olumlu"]]
        neg = [normalize_text(w) for w in SENTIMENT_KEYWORDS["Olumsuz"]]
        positive = sum(1 for word in words if word in pos)
        negative = sum(1 for word in words if word in neg)


        if positive > negative:
            return "Olumlu"
        elif negative > positive:
            return "Olumsuz"
        else:
            return "Nötr"
    except Exception as e:
        print(f"⚠️ Sentiment analiz hatası: {str(e)}")
        return "Nötr"



def analyze_review(text):
    result = {aspect: "Nötr" for aspect in ASPECTS_KEYWORDS}
    text = str(text).lower()
    for aspect, keywords in ASPECTS_KEYWORDS.items():
        for keyword in keywords:
            if keyword_match(text, keyword):
                sentiment = detect_sentiment(text, keyword)
                if sentiment != "Nötr":
                    result[aspect] = sentiment
                    break
    return result

In [52]:
# --- VERİ YÜKLE ---
df = pd.read_csv(INPUT_PATH, usecols=["app_id", "review_text"], dtype={"app_id": str, "review_text": str}).dropna()

In [53]:
df.head()

,app_id,review_text
0,1245620,İlk 40 saatimde nereye gitmem gerektiğini ne y...
1,1245620,Bu oyunda Malenia bossunu tasarlıyan arkadaşa ...
2,1245620,Güzel oyun atmosfer kontroller vs. güzel ama ...
3,1245620,Oyunun devasa bir haritası var açık dünya olma...
4,1245620,oynu bitirdiğinizde huzurluca ölebilirisiniz


In [54]:
# --- ANALİZ ---
results = []
for index, row in df.iterrows():
    analysis = analyze_review(row["review_text"])
    results.append({
        "app_id": row["app_id"],
        "review_text": row["review_text"],
        **analysis
    })
df_results = pd.DataFrame(results)


In [ ]:
# --- RENKLİ KAYDET ---
with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl") as writer:
    df_results.to_excel(writer, index=False, sheet_name="Analizler")
    worksheet = writer.sheets["Analizler"]
    for col in range(3, 6):
        col_letter = get_column_letter(col)
        for row in range(2, len(df_results) + 2):
            cell = worksheet[f"{col_letter}{row}"]
            cell.fill = COLOR_MAP.get(cell.value, COLOR_MAP["Nötr"])
    worksheet.column_dimensions['A'].width = 15
    worksheet.column_dimensions['B'].width = 80


# --- ÖZET SHEET ---
summary_data = []
for aspect in ASPECTS_KEYWORDS:
    counts = df_results[aspect].value_counts().to_dict()
    summary_data.append({
        "Kategori": aspect,
        "Olumlu": counts.get("Olumlu", 0),
        "Olumsuz": counts.get("Olumsuz", 0),
        "Nötr": counts.get("Nötr", 0)
    })
df_summary = pd.DataFrame(summary_data)
with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl", mode="a") as writer:
    df_summary.to_excel(writer, index=False, sheet_name="Özet")


# --- NÖTR KELİMELER ---
neutral_reviews = df_results[
    (df_results["Grafik"] == "Nötr") &
    (df_results["AI"] == "Nötr") &
    (df_results["Oynanis"] == "Nötr") & 
    (df_results["Ses ve Muzik"] == "Nötr") &
    (df_results["Oyun Dunyasi"] == "Nötr") &
    (df_results["Topluluk ve Sosyal"] == "Nötr") &
    (df_results["Hikaye ve Senaryo"] == "Nötr") &
    (df_results["Performans ve Teknik"] == "Nötr") 
]["review_text"].tolist()
neutral_corpus = " ".join(map(str, neutral_reviews)).lower()
neutral_corpus = normalize_text(neutral_corpus)
neutral_words = re.findall(r'\b\w{3,}\b', neutral_corpus)
most_common_words = Counter(neutral_words).most_common(100)
df_top_words = pd.DataFrame(most_common_words, columns=["Kelime", "Frekans"])
with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl", mode="a") as writer:
    df_top_words.to_excel(writer, index=False, sheet_name="Nötr En Çok Kelimeler")

    
print(f"✅ İşlem tamamlandı! Excel çıktısı: {OUTPUT_PATH}")

KeyError: 'AI'